# init

In [1]:
# Use a database already created on mongolab 
server = 'ds059694.mongolab.com'
port = 59694
db_name = 'deepreader'
username = 'deepreaderuser'
password = '1ecolequimonte45'

from pymongo import MongoClient as Connection

# from pymongo import Connection

# what versions are we using
import sys
print 'Python version', sys.version

import pymongo
print 'Pymongo version', pymongo.version
##

# connect to server
print '\nConnecting ...'
conn = Connection(server, port)

# Get the database
print '\nGetting database ...'
db = conn[db_name]

# Have to authenticate to get access
print '\nAuthenticating ...'
db.authenticate(username, password)
# from pymongo import MongoClient
# client = MongoClient()
# db = client.izidb


Python version 2.7.10 |Anaconda 2.3.0 (64-bit)| (default, Nov  7 2015, 13:18:40) [MSC v.1500 64 bit (AMD64)]
Pymongo version 3.1.1

Connecting ...

Getting database ...

Authenticating ...


True

In [2]:
from scripts import izi
reload(izi)

n_topics = 100

--------------------> english word frequencies loaded
--------------------> english word frequencies loaded


# write LDA topics to file


In [3]:
# izi.saveTopicsAsTxt(u'../masc/0LDA_TOPIC/')

# get files list

In [4]:
root = u"../izi_data/"
filelist = izi.getFileList(root)
print len(filelist)
example =  filelist[0]
print example[len(root):]

195
1412 Standard English Review-en-ru-T.mxliff


In [5]:
def safe_str(value):
    if type(value) == str:
    # Ignore errors even if the string is not proper UTF-8 or has
    # broken marker bytes.
    # Python built-in function unicode() can do this.
        value = unicode(value, "utf-8", errors="ignore")
    else:
    # Assume the value object has proper __unicode__() method
        value = unicode(value)
    return value

# define functions

In [6]:
from bs4 import BeautifulSoup
def loadTranslator(path):   
    if path[-4:] == 'liff':
        soup = BeautifulSoup( open(path), 'lxml')
        for string in soup.find_all("file"):
            try:
                return string['m:task-id']
            except:
                return -1
            
            
def insertDoc(path, root = root, prefix = ''):
    document = dict()
    document['title'] = prefix + '___' + path[len(root):].replace( ' ', '_')
#     document['title'] = path[len(root):].replace( ' ', '_')
    document['type'] = prefix
    document['translator'] = loadTranslator(path)
    current_translator = db.translators.find_one({'name': document['translator']})
        
        
    try:
        full_text = izi.loadText(path)
    #     full_text = unicode(full_text, errors='ignore')
        full_text = safe_str(full_text)

        if len(full_text) > 500:
            if not current_translator:
                translator = dict()
                translator['name'] = document['translator']
                translator['document'] = []
                id_current_translator = db.translators.save(translator)
                current_translator = db.translators.find_one({'name': document['translator']})

            # text and tokens
            tokens = izi.tokenize( full_text)
            document['full_text'] = full_text
            document['tokens'] = tokens
            # topics
            topics =  izi.topicsFromTokens(izi.tokenize(full_text))
            semantic_vec = [0.] * n_topics
            for i in topics:
                semantic_vec[i[0]] = i[1]
            document['semantic_vec'] = semantic_vec
            # complexity
            document['complexity'] = izi.complexityAlongtheText(full_text)
            # topic distribution
            document['full_topics'] = izi.getTopicDistributionData( document['full_text'], document['semantic_vec'])
            # significants words
            document['significantWords'] = izi.getMostSignificantWordsData(document['tokens'] , document['semantic_vec'])
            # significant words graph
            document['topicsGraph'] = izi.SignificantWordsGraph(document['tokens'] , document['semantic_vec'] )

            ##
            # savec doc
            current_id = db.documents.save(document)
            current_translator['document'].append(current_id)
            db.translators.update({'name': document['translator']}, {'name': document['translator'],\
                                                                     'document': current_translator['document']})


            ###################
            # create links
            cursor = db.documents.find()
            for doc in cursor:
                y = doc['semantic_vec']
                y_id = doc["_id"]
                if y_id != current_id:
                    s = izi.getSimilarity( semantic_vec, y)
                    db.similarities.insert({'source': current_id , 'target': y_id, 'value': s})
    except:
        print "############################################################"
        print "#### ERROR: " + document['title']

In [7]:
class idGenerator:
	def __init__(self):
		self.id = 0
	def get(self):
		self.id += 1
		return self.id - 1
    
def getLastAdded():
	current_id = None
	for d in db.documents.find().sort("_id", -1).limit(1) :
		current_id = d
	return db.documents.find_one( { "_id" : current_id["_id"] })


def getGraph():
    SIMILARITY_CUTOFF = 0.85

    lasDoc = getLastAdded()
    semantic_vectors = dict()
    list_ids = db.documents.find().distinct("_id")
    current_id = lasDoc["_id"]

    id2db = dict()
    gen = idGenerator()

    nodes = []
    for ii in list_ids:
        doc = db.documents.find_one( {'_id' : ii} )
        node = dict()
        tmp_id = doc["_id"]
        i = gen.get()
        id2db[str(tmp_id)] = i
        node["id"] = i
        print doc['type']
        if doc['type'] == '0LDA_TOPIC':
            node['color'] = "#aaaaaa"
            node['size'] = 8
        else:
            node['color'] = "#555555"
            node['size'] = 5
        node['id_db'] = str(tmp_id)
        node['name'] = doc['title']
        nodes.append(node)

    cursor = db.similarities.find()
    edges = []
    for e in cursor:
        if e['value'] > SIMILARITY_CUTOFF:
            a = e['source']
            b = e['target']
            edge = {'source': id2db[str(a)] , 'target': id2db[str(b)], 'value': e['value']}
            edges.append(edge)


    graph = dict()
    graph['nodes'] = nodes
    graph['links'] = edges

    return graph

# Brown

In [ ]:
root2 = u"../brown/"
import nltk
brown_ids = nltk.corpus.brown.fileids()
for i in brown_ids:
    f = open( root2 + i + ".txt", "w")
    f.write(nltk.corpus.brown.raw(i) )

In [ ]:
for f in izi.getFileList(root2):
    insertDoc(f, root = root2)

In [ ]:
# cursor = db.documents.find_one({ "_id": id})
# for document in cursor:
#     print document

# scripts

In [ ]:
import urllib
root_scripts = u"../scripts_movies/"


In [ ]:
f = open('raw_script_urls.txt', 'r')
for l in f.readlines():
    ll =  [ u.rstrip() for u in l.split(' +++$+++ ') ]
    title = ll[1]
    title = title.replace(" ", "_")
    link = ll[2]
    try:
        p = urllib.urlopen(link).read()
        if len(p) > 1000:
            print link + ' ' + title
            path = root_scripts + title + ".txt"
            file = open( path, "w")
            
            file.write( p )
            insertDoc(path, root = root_scripts)
        else:
            print 'TOO SHORT: ' + link + ' ' + title
    except:
        print "ERROR: " + title

In [ ]:
print izi.getFileList(root_scripts)

In [ ]:
fails = []
print len(izi.getFileList(root_scripts))
i = 0
for f in izi.getFileList(root_scripts):
    try:
        insertDoc(f, root = root_scripts)
        print i,
    except:
        fails.append(f)
        print "#",
    i += 1

# masc

In [9]:
root_masc = u"../masc/"
k = 0
limit = 4
for f in izi.getFileList(root_masc):
    sub_root = f+'/'
    for ff in izi.getFileList(sub_root):
        print str(k) + '  |   ' +  str(ff)
        if True: #k < limit:
            insertDoc(ff, root = sub_root, prefix = f[len(root_masc):])
            k+=1
        else:
            break

0  |   ../masc/0LDA_TOPIC/12.txt
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:31: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:55: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead



Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
1  |   ../masc/0LDA_TOPIC/13.txt
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence 

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:57: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:68: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.



cannot compute complexity in 'complexityAlongtheText' 
2  |   ../masc/0LDA_TOPIC/15.txt
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute complexity in 'complexityAlongtheText' 
Error(ASL): Sentence Count is Zero, Cannot Divide
cannot compute comple

In [ ]:
db.graph.insert(  getGraph() )

# add all documents

In [ ]:
from scripts import izi
reload(izi)

In [ ]:
# fails = []
# i = 0
# maxx = 5
# for f in filelist:
#     if True: #i < maxx:
#         try:
#             print f
#             insertDoc(f)
#         except:
#             fails.append(f)
#             print "#"

#         i += 1
    
# print
# print
# print "%s fails" %len(fails)

In [ ]:
for f in fails:
    print f

# tests

In [ ]:
cursor = db.documents.find_one()
for document in cursor:
    print document
    
    
x = cursor["_id"]

db.documents.find_one({ "_id": x})["full_text"]

In [ ]:
db.similarities.find_one()

In [ ]:
db.translators.find_one()

# reset collections

In [8]:
db.documents.drop()
db.similarities.drop()
db.graph.drop()
db.translators.drop()